# AuxTel accelerometer PSD - 29-Sep-22
Diagnosing bad accelerometer channel \
Craig Lage

In [ ]:
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst_efd_client.efd_helper import EfdClient
#from lsst_efd_client.efd_utils import merge_packed_PSD

In [ ]:
# Get EFD client
client = EfdClient('summit_efd')

In [ ]:
# Just using this in the notebook for now.
def merge_packed_PSD(packed_dataframe, base_field, sensor_names):
    """Select fields that represent the Power Spectral Density of \                                                          
    a sensor and unpack them into a dataframe with PSD vs frequency.                                                         
    Parameters                                                                                                               
    ----------                                                                                                               
    packed_dataframe : `pandas.DataFrame`                                                                                    
        packed data frame containing the desired data                                                                        
    base_field :  `str`                                                                                                      
        Base field name that will be expanded to query all                                                                   
        vector entries.                                                                                                      
    sensor_name :  `str` or list                                                                                             
        Name of the sensor(s) of interest.                                                                                   
    Returns                                                                                                                  
    -------                                                                                                                  
    result : `pandas.DataFrame`                                                                                              
        A `pandas.DataFrame` containing the results of the query.                                                            
    """
    minPSDFrequency = packed_dataframe['minPSDFrequency'][0]
    maxPSDFrequency = packed_dataframe['maxPSDFrequency'][0]
    numDataPoints = packed_dataframe['numDataPoints'][0]
    if isinstance(sensor_names, str):
        sensor_names = [sensor_names, ]

    packed_dataframe = packed_dataframe.loc[packed_dataframe.sensorName.isin(sensor_names)]
    packed_fields = [k for k in packed_dataframe.keys()
                     if k.startswith(base_field) and k[len(base_field):].isdigit()]
    packed_fields = sorted(packed_fields, key=lambda k: int(k[len(base_field):]))  # sort by pack ID                         
    npack = len(packed_fields)
    assert numDataPoints == npack, "Number of packed data points does not match numDataPoints!"
    packed_len = len(packed_dataframe)
    output = np.empty(npack * packed_len)
    deltaF = float(maxPSDFrequency - minPSDFrequency) / (npack - 1) # Frequency step                                         
    columns = []
    for i in range(npack):
        label = f"{base_field}{i}"
        columns.append(minPSDFrequency + i * deltaF)
        output[i::npack] = packed_dataframe[label]
    output = np.reshape(output, (packed_len, npack))
    return pd.DataFrame(data=output, columns=columns, index=packed_dataframe.index)

# This dictionary defines the axis scramble
trueAxes = {'AuxTel-M1':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-M2':{'X':'El', 'Y':'Az', 'Z':'Opt'}, \
            'AuxTel-Truss':{'X':'El', 'Y':'Opt', 'Z':'Az'}}


In [ ]:
# Times to start looking at PSD data
start = Time("2022-10-04 13:24:40Z", scale='utc')
end = Time("2022-10-04 13:24:60Z", scale='utc')
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)
axis='X'
sensor = "AuxTel-Truss"
plotCounter = 1
base_field = f"accelerationPSD{axis}"
df = merge_packed_PSD(accel_data, base_field, sensor)

times = df.index.to_list()
freqs = df.columns

centerFreq = 10.0
deltaFreq = 0.2 # Plus/minus 20%
minFreq = centerFreq * (1.0 - deltaFreq)
maxFreq = centerFreq * (1.0 + deltaFreq)
plotTimes = []
sums = []
for i in range(len(df)):
    plotTimes.append(times[i])
    PSD_vals = df.iloc[i].to_list()
    sum = 0.0
    for j,f in enumerate(freqs):
        if f > minFreq and f < maxFreq:
            sum += PSD_vals[j]
    sums.append(sum)
plt.scatter(plotTimes, sums)

In [ ]:
# Times to start looking at PSD data
start = Time("2022-10-04 13:26:00Z", scale='utc')
end = Time("2022-10-04 13:26:30Z", scale='utc')
accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)
axis='Z'
sensor = "AuxTel-Truss"
plotCounter = 1
base_field = f"accelerationPSD{axis}"
df = merge_packed_PSD(accel_data, base_field, sensor)

times = df.index.to_list()
freqs = df.columns

centerFreq = 68.0
deltaFreq = 0.2 # Plus/minus 20%
minFreq = centerFreq * (1.0 - deltaFreq)
maxFreq = centerFreq * (1.0 + deltaFreq)
plotTimes = []
sums = []
for i in range(len(df)):
    plotTimes.append(times[i])
    PSD_vals = df.iloc[i].to_list()
    sum = 0.0
    for j,f in enumerate(freqs):
        if f > minFreq and f < maxFreq:
            sum += PSD_vals[j]
    sums.append(sum)
plt.scatter(plotTimes, sums)

In [ ]:
peaks = [["AuxTel-M2", "Z", 10.0], ["AuxTel-Truss", "X", 10.0], ["AuxTel-Truss", "Y", 10.0], \
         ["AuxTel-Truss", "Z", 68.0]]

start = Time("2022-09-29 22:00:00Z", scale='utc')
end = Time("2022-09-30 04:00:00Z", scale='utc')
#accel_data = await client.select_time_series("lsst.sal.ESS.accelerometerPSD", ["*"], start, end)
plotCounter = 1
plt.figure(figsize=(8,8))
plt.suptitle("Time Evolution of AuxTel Accelerometer Peaks 2022-09-29")
for [sensor, axis, centerFreq] in peaks:
    base_field = f"accelerationPSD{axis}"
    trueAxis = trueAxes[sensor][axis]
    df = merge_packed_PSD(accel_data, base_field, sensor)
    times = df.index.to_list()
    freqs = df.columns
    deltaFreq = 0.2 # Plus/minus 20%
    minFreq = centerFreq * (1.0 - deltaFreq)
    maxFreq = centerFreq * (1.0 + deltaFreq)
    plotTimes = []
    sums = []
    for i in range(len(df)):
        plotTime = (times[i].day-29)*24.0 + times[i].hour + times[i].minute/60.0 + times[i].second/3600.0
        plotTimes.append(plotTime)
        PSD_vals = df.iloc[i].to_list()
        sum = 0.0
        for j,f in enumerate(freqs):
            if f > minFreq and f < maxFreq:
                sum += PSD_vals[j]
        sums.append(sum)
        #if sensor=='AuxTel-Truss' and trueAxis=='El' and sum>5.0E-9:
        #    print(plotTimes[-1], sum)
    plt.subplot(2,2,plotCounter)
    plt.title(f"{sensor}-{trueAxis}-{centerFreq} Hz")
    plt.scatter(plotTimes, sums)
    plt.ylim(0,50.0E-10)
    plotCounter += 1
plt.savefig("/home/craiglagegit/DATA/Accel_PSD_Time_Evolution_30Sep22.pdf")

In [ ]:
plotTimes

In [ ]:
test = plotTimes[-1]

In [ ]:
test.hour + test.minute / 60.0 + test.second / 3600

In [ ]:
i=5000

In [ ]:

(times[i].day-29)*24.0 + times[i].hour + times[i].minute/60.0 + times[i].second/3600.0

In [ ]:
times[5000].day